## 딥러닝 모델 데이터 타입

- 컴퓨터에서는 일반적으로 소수 연산을 위해 32비트 부동소수점(float32)사용
- 더 세밀한 계산이 필요하면 64비트 부동소수점(float64)을 사용
- 최근에는 성능 유지하면서 더 적은 비트의 데이터 타입을 사용하는 방식 -> float16 or brain float 16

## 양자화(Quantization)
- fp32를 fp16으로 저장하면 모델의 용량은 절반이 되지만 세밀한 표현은 불가하다. 
    - ex. 7B(Billion) 모델이 16비트(2바이트) 데이터 형식으로 저장되면 용량은 7 * 2 = 14GB
- 양자화에서는 더 적은 비트를 사용하면서도 원본 데이터의 정보를 최대한 소실 없이 유지하는 것이 핵심 과제이다.

## 양자화 방식
1. MinMaxQuantization: float32의 데이터를 int8로 변환할 때, float32 크기 자체의 Max/Min의 값을 이용하여 int의 양끝에 매칭시키는 방식. 이 방식은 양쪽 끝에는 사용하는 데이터가 존재하지 않아 낭비되는 문제 발생

2. AbsoluteMinMaxQuantization: float32 자체의 최대 최소 값 말고, float32안에 존재하는 데이터의 Min/Max의 절댓값을 int8의 최대 최소로 정의하면 낭비되는 문제를 해결할 수 있음. 하지만 이 방식도 이상치가 존재하는 경우 낭비되는 문제 발생 가능성 O

3. Quantile 방식: 절대 최댓값만(absmax)만 보는 것이 아니라 입력 데이터를 크기 순으로 등수를 매겨 int8 값에 동일한 개수의 fp32 값이 대응되도록 배치하는 방식 -> 별도의 메모리를 사용해야함

In [ ]:
## 메모리 측정 함수 구현 
import torch
import transformers
import datasets
import peft
import bitsandbytes
import warnings
warnings.filterwarnings('ignore')

def print_gpu_utilization():
    if torch.cuda.is_available():
        used_memory = torch.cuda.memory_allocated() / 1024 * 3
        print(f"GPU 메모리 사용량: {used_memory:.3f} GB")
    else:
        print("사용중인 GPU 없음")

print_gpu_utilization()

## 분산 학습
- 데이터 병렬화(data parallelism): 하나의 GPU에올릴 수 있는 경우 여러 GPU에 각각 모델을 올리고 학습 데이터를 병렬로 처리해 학습 속도를 높일 수 있음

- 모델 병렬화(model parallelism): 하나의 GPU에 올리기 어려운 큰 모델의 경우 층별로 나눠 GPU에 올리는 파이프라인 병렬화(pipeline parallelism)와 한 층의 모델도 나눠서 올리는 텐서 병렬화(tensor parallelism)이 있다.

## ZeRO(Zero Redundancy Optimizer)
- 분산 학습에서 동일한 모델을 여러 GPU에 올려 중복으로 메모리를 차지하게 되는데, 메모리 관리 측면에서는 비효율적이다. ZeRO는 모델을 나눠 여러 GPU에 올리고 각 GPU에서는 자신의 모델 부분의 연사만 수행하고 그 상태를 메모리에 저장하여 효율적이고 속도도 빠르게 유지할 수 있다. 

## PEFT(Parameter Efficient Fine-Tuning) & LoRA(Low-Rank Adaptation)
- 일부 파라미터만 학습하는 방법
- LoRA는 모델에 일부 파라미터를 추가하고 그 부분만 학습하는 방식이다. 랭렬을 더 작은 2개의 행렬의 곱으로 표현하여 전체 파라미터를 수정하는 것이 아니라 더 작은 2개의 행렬을 수정하는 것을 의미한다. 
    - d * d인 모델 전체파라미터를 A(d * r)과 B(r * d)의 곱으로 나누어 동일한 효과를 낼 수 있다. 
    - 행렬이 조금 추가되지만, W의 Gradient와 Optimizer 상태 등을 저장하는 메모리가 줄어드므로 효율적이다.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model # lora model load

def load_model_and_tokenizer(model_ckpt, peft = None):
    tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
    
    if peft is None:
        model = AutoModelForCausalLM.from_pretrained(model_ckpt, torch_dtype = 'auto', device_map = {"":0})
    
    elif peft == 'lora':
        # 모델의 가중치를 자동으로 적절한 데이터 타입으로 로드
        model = AutoModelForCausalLM.from_pretrained(model_ckpt, torch_dtype = 'auto', device_map = {"":0})
        lora_config = LoraConfig(
            r = 8, # 행렬의 랭크(rank) -> 값이 작을수록 학습 가능한 파라미터 줄어듬 
            lora_alpha = 16, # LoRA 가중치 스케일링 벡터; 보통 r * 2로 설정 
            target_modules = ["query_key_value"], # LoRA를 적용할 모델 내의 특정 레이어 이름; 여기서는 어텐션 메커니즘 연산에 적용 
            lora_dropout = 0.05, 
            bias = "none",
            task_type = "CAUSAL_LM" 
        )
        
        # 모델에 LoRA 어댑터 주입하여 새로운 PEFT 모델 반환
        model = get_peft_model(model, lora_config)
        model.print_trainable_parameters()
        
    print_gpu_utilization()
    return model, tokenizer

